## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [3]:
data = range(1,1001)
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [1]:
import pyspark
conf = pyspark.SparkConf().set('spark.driver.host','127.0.0.1')
sc = pyspark.SparkContext('local[*]', conf=conf)

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [9]:
# first
rdd.first()

1

In [10]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [11]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [14]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [15]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([1.48102313e-01, 2.86766309e-01, 7.21385754e-01, 3.57894695e+00,
       4.70945327e+00, 5.05313125e+00, 2.14321477e+00, 3.50141637e-01,
       3.10805234e+00, 7.74572908e+00, 5.29992488e+00, 5.42334334e+00,
       9.17413314e+00, 1.15305178e+01, 7.31832794e+00, 5.81281289e+00,
       9.68823011e+00, 1.37979888e+01, 1.13922557e+01, 1.98465644e+01,
       4.61926025e-01, 1.42197712e+01, 6.30313991e+00, 7.38218609e+00,
       2.25496701e+01, 2.18362073e+01, 1.62493672e+01, 2.16849028e+01,
       6.45109845e+00, 1.37719137e+01, 1.78997169e+01, 3.14385229e+01,
       3.27144112e+01, 6.48688743e+00, 2.98365263e+01, 4.74530817e+00,
       1.58804554e+01, 9.85915005e+00, 4.85357577e+00, 2.92799884e+01,
       3.03686068e+01, 2.73353299e+00, 1.58817648e+01, 3.41651073e+01,
       1.19252559e+01, 4.40221620e+01, 3.16340870e+01, 3.50926338e+01,
       3.30482994e+01, 3.45171919e+01, 2.99929481e+01, 1.29271755e+01,
       1.15945587e+01, 2.92512796e+00, 4.78430989e+01, 1.11728723e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [18]:
price_items = sc.parallelize(sales_figures, 10)
price_items.top(5)

[954.4924791246783,
 940.5909588583698,
 893.9193912704777,
 883.6761603823927,
 878.5293938949981]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [20]:
def sales_tax(num):
    profit_after_tax = num - (num * .08)
    return profit_after_tax

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [22]:
# perform action to retrieve rdd values
revenue_minus_tax.top(5)

[878.133080794704,
 865.3436821497003,
 822.4058399688395,
 812.9820675518013,
 808.2470423833983]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [25]:
discounted = revenue_minus_tax.map(lambda x: x - (x*.1))

In [27]:
discounted.take(10)
discounted.top(5)

[790.3197727152336,
 778.8093139347302,
 740.1652559719555,
 731.6838607966212,
 727.4223381450585]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [33]:
price_items.map(sales_tax).map(lambda x : x - (x*0.1)).top(5)

[790.3197727152336,
 778.8093139347302,
 740.1652559719555,
 731.6838607966212,
 727.4223381450585]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [35]:
discounted.toDebugString()

b'(10) PythonRDD[20] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[10] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [40]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.14810231290385545, 0.12262871508439233), (0.2867663088055792, 0.23744250369101957), (0.7213857543632629, 0.5973074046127818), (3.578946949672449, 2.963368074328788), (4.709453272487551, 3.899427309619692), (5.053131245937302, 4.183992671636086), (2.143214774430678, 1.7745818332286016), (0.3501416367450956, 0.28991727522493915), (3.1080523357800223, 2.5734673340258585), (7.745729078075864, 6.413463676646816)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [42]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.14810231290385545, 0.12262871508439233, 0.2867663088055792, 0.23744250369101957, 0.7213857543632629, 0.5973074046127818, 3.578946949672449, 2.963368074328788, 4.709453272487551, 3.899427309619692]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [52]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.first()
selected_items.count()

271

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [53]:
selected_items.reduce(lambda x,y :x + y)

125239.20459897623

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [56]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(10)

[(31, 314.98808188329616),
 (28, 320.401634932181),
 (41, 326.5087767651979),
 (15, 334.88416685221716),
 (9, 345.4126361138794),
 (48, 300.72591806785255),
 (39, 307.2009413823908),
 (26, 331.60377559216056),
 (42, 337.69605547872084),
 (33, 301.55701741537644)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [57]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y :x + y)
total_spent.take(10)

[(30, 1343.853746445164),
 (20, 1533.8298163665818),
 (50, 2851.600315669196),
 (40, 2727.4207952215975),
 (10, 1529.164260302838),
 (11, 2254.8294276634406),
 (41, 646.5921462583128),
 (31, 3069.0237620432536),
 (1, 4014.8913657124267),
 (21, 1907.7504771216052)]

In [67]:
# sort the users from highest to lowest spenders
total_spent.sortBy((lambda a: a[1]), ascending=False).collect()


[(25, 5215.265738940392),
 (26, 5029.893832652313),
 (4, 4088.708729164231),
 (1, 4014.8913657124267),
 (16, 3997.48450949696),
 (8, 3856.039413826153),
 (9, 3566.139373505739),
 (45, 3456.117421770016),
 (28, 3438.0179289753414),
 (38, 3421.0989595363435),
 (12, 3321.8064804728406),
 (15, 3192.373740182544),
 (36, 3103.849426773685),
 (31, 3069.0237620432536),
 (37, 3044.9364432139264),
 (29, 3038.927524977139),
 (24, 3034.87309229206),
 (34, 3032.1519591621245),
 (50, 2851.600315669196),
 (42, 2833.388121884712),
 (49, 2812.766660998989),
 (40, 2727.4207952215975),
 (35, 2697.872198672583),
 (32, 2666.071154596092),
 (43, 2642.954511353053),
 (14, 2534.0194941894893),
 (17, 2418.6729785290763),
 (19, 2350.4282638584236),
 (11, 2254.8294276634406),
 (6, 2185.7271458942046),
 (47, 2163.726902409495),
 (5, 2083.5482687244557),
 (27, 1999.7492295373613),
 (2, 1931.9482930054326),
 (21, 1907.7504771216052),
 (3, 1871.8610412714838),
 (7, 1678.3966380294492),
 (20, 1533.8298163665818),
 (3

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [68]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(39, 12),
 (50, 10),
 (24, 9),
 (13, 8),
 (48, 8),
 (25, 8),
 (31, 8),
 (45, 7),
 (5, 7),
 (29, 7),
 (1, 7),
 (3, 7),
 (46, 7),
 (22, 7),
 (19, 7),
 (14, 7),
 (47, 6),
 (37, 6),
 (43, 6),
 (8, 6),
 (16, 6),
 (12, 6),
 (23, 6),
 (32, 6),
 (33, 6),
 (27, 6),
 (28, 5),
 (18, 5),
 (6, 5),
 (15, 5),
 (34, 5),
 (36, 5),
 (40, 4),
 (11, 4),
 (4, 4),
 (30, 4),
 (9, 4),
 (10, 4),
 (44, 4),
 (35, 4),
 (42, 4),
 (20, 4),
 (7, 3),
 (26, 3),
 (49, 3),
 (38, 2),
 (41, 1),
 (2, 1),
 (21, 1),
 (17, 1)]

In [3]:
sc.stop()

In [4]:
sc

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local[*] appName=pyspark-shell>

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
